# 09wk-1: 추천시스템

최규빈  
2024-04-28

In [2]:
import torch
import numpy as np 
import pandas as pd
import fastai.collab

In [3]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df_view 

In [4]:
a1 = np.array([1.9,0.0]).reshape(2,1) 
b1= np.array([2.0,0.1]).reshape(2,1)

In [5]:
sum(a1*b1)

In [6]:
a1= np.array([1.9,0.0]).reshape(2,1)
a2= np.array([2.0,0.1]).reshape(2,1)
a3= np.array([2.5,1.0]).reshape(2,1)
a4= np.array([0.1,1.9]).reshape(2,1)
a5= np.array([0.2,2.1]).reshape(2,1)
a6= np.array([0.0,1.5]).reshape(2,1)
A = np.concatenate([a1,a2,a3,a4,a5,a6],axis=1)
A

In [7]:
b1= np.array([2.0,0.1]).reshape(2,1)
b2= np.array([1.9,0.2]).reshape(2,1)
b3= np.array([1.8,0.3]).reshape(2,1)
b4= np.array([0.3,2.1]).reshape(2,1)
b5= np.array([0.2,2.0]).reshape(2,1)
b6= np.array([0.1,1.9]).reshape(2,1)
B = np.concatenate([b1,b2,b3,b4,b5,b6],axis=1)
B

In [8]:
A.T@B

In [9]:
a1.T@b1, a2.T@b2, a3.T@b1

In [10]:
A.T@B 

In [11]:
df_view

In [12]:
df = pd.DataFrame([(f,m,df_view.loc[f,m]) for f in df_view.index for m in df_view.columns if not np.isnan(df_view.loc[f,m])])
df.columns = ['X1','X2','y']
df

In [13]:
mapp1 = {k[1]:k[0] for k in enumerate(df.X1.unique())}
mapp2 = {k[1]:k[0] for k in enumerate(df.X2.unique())}
mapp1,mapp2

In [38]:
X1 = torch.tensor(list(map(lambda name: mapp1[name], df.X1)))
X2 = torch.tensor(list(map(lambda name: mapp2[name], df.X2)))
X1_onehot = torch.nn.functional.one_hot(X1).float()
X2_onehot = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df.y).float()

In [39]:
X1,X2,y

In [40]:
X1_ebdd

In [41]:
e1 = torch.nn.Embedding(6,2)
l1 = torch.nn.Linear(in_features=6,out_features=2) 

In [42]:
ebdd(X1),l1(X1_onehot)

In [ ]:
li